In [1]:
# !pip install pytumblr
# !pip install requests-cache
# !pip install pymongo

In [2]:
# !rm web_cache.sqlite

In [3]:
import requests_cache
requests_cache.install_cache('web_cache')

In [4]:
import pytumblr
import json
from pymongo import MongoClient
from tqdm.notebook import tqdm
from IPython.display import JSON

In [5]:
mongo = MongoClient('172.17.0.1', 27017)
db = mongo['bad-vis']
raw = db['tumblr-raw']

In [6]:
credentials = ""

with open('./tumblr_credentials.json') as f:
    credentials = json.load(f)

In [7]:
client = pytumblr.TumblrRestClient(credentials['consumer_key'])

In [8]:
JSON(client.posts('badvisualisations.tumblr.com', limit=1))

<IPython.core.display.JSON object>

In [9]:
reply = client.posts('badvisualisations.tumblr.com', limit=100)

In [10]:
len(reply['posts'])

50

In [11]:
JSON(reply['posts'][0])

<IPython.core.display.JSON object>

In [12]:
def digest_post (post):
    if not raw.find_one({'id': post['id']}, {'_id': 1}):
        return raw.insert_one(post)
    else:
        return ''

def retrieve (blog, offset=0, limit=50):
    url = blog
    reply = client.posts(url, offset=offset, limit=limit)
    total = reply['total_posts']
    return [post for post in reply['posts']], total

def posts_iter (blog):
    offset = 0
    total = 1
    while offset < total:
        posts, total = retrieve(blog, offset)
        offset += len(posts)
        for post in posts:
            yield post

def retrieve_posts (blog):
    return (digest_post(p) for p in tqdm(posts_iter(blog)))

In [13]:
crawler_queue = [
    {'source': 'badvisualisations', 'file': 'badvisualisations.json', 'blog': 'badvisualisations.tumblr.com'},
    {'source': 'wtf-viz', 'file': 'wtf-viz.json', 'blog': 'viz.wtf'}
]

In [14]:
for c in crawler_queue:
    posts = [p for p in retrieve_posts(c['blog'])]